In [23]:
import numpy as np
import pandas as pd
import matplotlib as mpl
# mpl.use('Cairo')  # for saving SVGs that Affinity Designer can parse
import matplotlib.pyplot as plt
import seaborn as sns
import dill

import candas as can
import gumbi as gmb
from candas.lims import Librarian, library
import pathlib as pl

code_pth = pl.Path.cwd()  # for running in Jupyter
# code_pth = pl.Path(__file__)  # for running in terminal
fig_pth = code_pth.parent
data_pth = fig_pth / 'data'
graph_pth = fig_pth / 'graphics'
graph_pth.mkdir(exist_ok=True)

gen_pth = fig_pth / 'generated'
gen_pth.mkdir(exist_ok=True)

plt.style.use(str(can.style.breve))

%config InlineBackend.figure_format = 'retina'

In [24]:
from candas.lims import Librarian, library
from candas.lims.schema import *

lib = Librarian().load(library).open()


In [20]:
def list_all(table, by: str = None):
        """List all objects of a given type in the database.

        Parameters
        ----------
        table : SQLAlchemy table
            Class from :mod:`schema`
        by : str, optional
            Object key to sort results by

        Returns
        -------
        list of Base
            List of database objects
        """

        results = lib.Session().query(table).all()

        if by is not None:
            if not hasattr(table, by):
                raise KeyError(f"{table.__name__} has no attribute {by}")
        else:
            if hasattr(table, "name"):
                by = "name"
            else:
                by = "id"

        results = sorted(results, key=lambda x: getattr(x, by))

        return results

In [25]:
list_all(Fragment)

[<Fragment(id=55, name=S036.0, description=None, flanks=['Flank_036.0_l', 'Flank_036.0_r'])>,
 <Fragment(id=84, name=S036.1, description=None, flanks=['Flank_036.0_l', 'Flank_036.0_r'])>,
 <Fragment(id=57, name=S036.1b, description=None, flanks=['Flank_036.0_l', 'Flank_036.0_r'])>,
 <Fragment(id=85, name=S036.2, description=None, flanks=['Flank_036.0_l', 'Flank_036.2_r'])>,
 <Fragment(id=86, name=S036.3, description=None, flanks=['Flank_036.3_l', 'Flank_036.3_r'])>,
 <Fragment(id=87, name=S036.4, description=None, flanks=['Flank_036.0_l', 'Flank_036.0_r'])>,
 <Fragment(id=67, name=S036.5, description=None, flanks=['Flank_036.5_l', 'Flank_036.5_r'])>,
 <Fragment(id=112, name=S036.6, description=None, flanks=['Flank_036.5_l', 'Flank_036.5_r'])>,
 <Fragment(id=113, name=S036.7, description=None, flanks=['Flank_036.5_l', 'Flank_036.5_r'])>,
 <Fragment(id=114, name=S036.8, description=None, flanks=['Flank_036.5_l', 'Flank_036.8_r'])>,
 <Fragment(id=115, name=S037.01.01, description=None, fl

In [17]:
lib.get_by_name(Experiment, 'JG073A').wells[0].reaction